In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
base = pd.read_csv("BASE.txt",sep=";")
monedas = pd.read_csv("Monedas.txt", sep=";")

In [3]:
base.head()

,Fecha,Empresa,Moneda,Ingreso
0,01/01/2017,Empresa A,EURO,0.000137
1,01/02/2017,Empresa A,EURO,0.001744
2,01/03/2017,Empresa A,EURO,0.126475
3,01/04/2017,Empresa A,EURO,0.000118
4,01/05/2017,Empresa A,EURO,0.000000


In [4]:
print(base.Empresa.unique())

['Empresa A' 'Empesa B' 'Empresa C' 'EmpresaC']


In [5]:
base = base.replace({'Empresa': {'EmpresaC':'Empresa C','Empesa B':'Empresa B'}})

In [6]:
print(base.Empresa.unique())
base.shape

['Empresa A' 'Empresa B' 'Empresa C']


(3383, 4)

In [7]:
monedas.head()
#print(monedas.Moneda.unique())

,FECHA,USD,EURO
0,01/01/2017,3000.71,3136.64
1,02/01/2017,3000.71,3140.24
2,03/01/2017,2981.06,3095.83
3,04/01/2017,2965.36,3094.95
4,05/01/2017,2941.08,3088.43


In [8]:
base['Fecha'] = pd.to_datetime(base['Fecha'])
monedas['FECHA'] = pd.to_datetime(monedas['FECHA'])

In [9]:
index_date_b = base['Fecha']
index_date_m = monedas['FECHA']
base.set_index(index_date_b, inplace=True)
monedas.set_index(index_date_m, inplace=True)
base.rename(columns = {'Fecha':'Date'}, inplace = True) 
monedas.rename(columns = {'FECHA':'Date'}, inplace = True)

In [10]:
df_unido = pd.merge(base, monedas, on = 'Date', how='inner',left_index=True, right_index= True)
df_unido.head()
#df_unido.shape




,Date,Empresa,Moneda,Ingreso,USD,EURO
2017-01-01,2017-01-01,Empresa A,EURO,0.000137,3000.71,3136.64
2017-01-01,2017-01-01,Empresa B,EURO,0.000329,3000.71,3136.64
2017-01-01,2017-01-01,Empresa C,EURO,0.000082,3000.71,3136.64
2017-01-01,2017-01-01,Empresa A,PESO,183.766913,3000.71,3136.64
2017-01-01,2017-01-01,Empresa B,PESO,871.384057,3000.71,3136.64


In [15]:
df = df_unido
#df.reset_index(inplace=True)
Ingreso_t=[]
for i in range(len(df)):
    mon = df['Moneda'][i]
    ingreso = df['Ingreso'][i]
    usd = df['USD'][i]
    euro = df['EURO'][i]
    if mon=='USD':
        ing = ingreso*usd
    elif mon=='EURO':
        ing = ingreso*euro
    else:
        ing = ingreso
    Ingreso_t.append(ing)

df['Ingreso_Total'] = Ingreso_t
df['year'] = pd.DatetimeIndex(df['Date']).year
df['trim'] = pd.PeriodIndex(df['Date'],freq='Q')
df.loc['2017-01-01':'2017-01-01']

,Date,Empresa,Moneda,Ingreso,USD,EURO,Ingreso_Total,year,trim
2017-01-01,2017-01-01,Empresa A,EURO,0.000137,3000.71,3136.64,0.430001,2017,2017Q1
2017-01-01,2017-01-01,Empresa B,EURO,0.000329,3000.71,3136.64,1.032003,2017,2017Q1
2017-01-01,2017-01-01,Empresa C,EURO,0.000082,3000.71,3136.64,0.258001,2017,2017Q1
2017-01-01,2017-01-01,Empresa A,PESO,183.766913,3000.71,3136.64,183.766913,2017,2017Q1
2017-01-01,2017-01-01,Empresa B,PESO,871.384057,3000.71,3136.64,871.384057,2017,2017Q1
2017-01-01,2017-01-01,Empresa C,PESO,363.767596,3000.71,3136.64,363.767596,2017,2017Q1
2017-01-01,2017-01-01,Empresa A,USD,0.019537,3000.71,3136.64,58.623491,2017,2017Q1
2017-01-01,2017-01-01,Empresa B,USD,0.019798,3000.71,3136.64,59.406964,2017,2017Q1
2017-01-01,2017-01-01,Empresa C,USD,0.007048,3000.71,3136.64,21.149903,2017,2017Q1


## Literal a

Ingreso por empresa donde la moneda es dólar

In [16]:
pd.options.display.float_format="{:,.2f}".format
ing_dolar = df[df['Moneda']=='USD'].groupby(['Empresa','Moneda']).agg({'Ingreso':['sum'],
                                                                       'Ingreso_Total':['sum']
                                                                      })
ing_dolar.columns =['Ingreso_origen','Ingreso_Pesos']
ing_dolar = ing_dolar.reset_index()#.sort_values(by=['Empresa'], ascending=False)
ing_dolar

,Empresa,Moneda,Ingreso_origen,Ingreso_Pesos
0,Empresa A,USD,9.86,"28,310.75"
1,Empresa B,USD,15.72,"45,344.65"
2,Empresa C,USD,6.22,"17,975.47"


## Literal b

Ingreso total por moneda de la Empresa A

In [18]:
pd.options.display.float_format="{:,.2f}".format
ing_empa = df[df['Empresa']=='Empresa A'].groupby(['Moneda']).agg({'Ingreso':['sum'],
                                                                       'Ingreso_Total':['sum']
                                                                      })
ing_empa.columns =['Ingreso_origen','Ingreso_pesos']
ing_empa = ing_empa.sort_values(by=['Ingreso_origen'], ascending=False).reset_index()
ing_empa

,Moneda,Ingreso_origen,Ingreso_pesos
0,PESO,"71,668.07","71,668.07"
1,USD,9.86,"28,310.75"
2,EURO,0.58,"1,697.57"


## Literal c

Empresa con los menores ingresos en pesos semestre 01/01/2017 - 
30/06/2017

In [88]:
df_sem = df.loc['2017-01-01':'2017-06-29']
df_sem.shape
ing_sem_pesos = df_sem.groupby(['Empresa']).agg({'Ingreso_Total':['sum']})
ing_sem_pesos.columns =['Ingreso_pesos']
ing_sem_pesos


,Ingreso_pesos
Empresa,
Empresa A,"58,616.55"
Empresa B,"204,309.79"
Empresa C,"87,692.79"


## Literal d

Ingreso para todos los trimestres del año 2017 en PESOS de cada una de las empresas

In [23]:
ing_trim_pesos = df.groupby(['Empresa','trim']).agg({'Ingreso_Total':['sum']})
ing_trim_pesos.columns =['Ingreso_Total_pesos']
ing_trim_pesos

Ingreso_Total_pesos
Empresa   trim                       
Empresa A 2017Q1            40,935.40
          2017Q2            17,054.93
          2017Q3            15,938.75
          2017Q4            27,747.31
Empresa B 2017Q1           121,246.16
          2017Q2            83,766.16
          2017Q3            72,530.68
          2017Q4            98,205.41
Empresa C 2017Q1            50,864.92
          2017Q2            37,413.40
          2017Q3            29,732.70
          2017Q4            43,739.65

## Literal e

Ingreso acumulado 2017 en PESOS desagregado por empresa y moneda

In [24]:
ing_year_pesos = df.groupby(['year','Empresa','Moneda']).agg({'Ingreso_Total':['sum']})
ing_year_pesos.columns =['Ingreso_Total_pesos']
ing_year_pesos

Ingreso_Total_pesos
year Empresa   Moneda                     
2017 Empresa A EURO               1,697.57
               PESO              71,668.07
               USD               28,310.75
     Empresa B EURO               3,160.44
               PESO             327,243.32
               USD               45,344.65
     Empresa C EURO               1,037.17
               PESO             142,738.04
               USD               17,975.47